In [1]:
import time
import datetime
import requests
import csv

In [2]:
#!pip install tqdm
from tqdm import notebook

In [3]:
session = requests.Session()
with open ("token.txt", "r") as f:
    x = f.readline()
session.headers.update({"Authorization": "token "+x.strip()})

In [4]:
error_url = []

def FetchDataFromGitHubRestApi(url):
    response = session.get(url)
    if response.status_code == 200:
        fetched_data = response.json()
        poll_interval = None
        poll_interval = response.headers.get('X-Poll-Interval')
        if poll_interval:
            time.sleep(int(poll_interval))
        return fetched_data
    else:
        error_url.append(url)

def Select(field, fetched_data):
    # Select Field From Fetched Data
    if fetched_data[field]:
        if type(fetched_data[field]) == str:
            try:
                output = str(fetched_data[field])
            except:
                output = fetched_data[field]
        else:
            output = str(fetched_data[field])
        output = output.replace("'",'').replace(", ",'/').replace(",",'/').replace('[','').replace(']','')
        if output == " " or output == "":
            output = "None"
    else:
        output = "None"
    return output

In [5]:
with open ("8K_github_username.txt", "r", encoding="utf8") as f:
    all_username = f.readlines()

In [6]:
with open ("not_a_user_or_no_data.txt", "r", encoding="utf8") as f:
    not_a_user_or_no_data = f.readlines()

In [7]:
print(len(all_username), len(not_a_user_or_no_data))

8932 349


In [8]:
for user in not_a_user_or_no_data:
    all_username.remove(user)
print(len(all_username))

8583


In [9]:
all_username = [x.strip() for x in all_username]

In [10]:
user_alias = ["usr"+str(c) for c in range(len(all_username))]

In [11]:
def CollectUserEventData(event_url):

    all_event_data = []
    fetched_event_data = FetchDataFromGitHubRestApi(event_url)

    for event in fetched_event_data:
        try:
            type_ = Select("type", event)
            created_at = Select("created_at", event)
            all_event_data.append([type_, created_at])
        except:
            pass

    return all_event_data

In [12]:
print("now ",datetime.datetime.now().hour,":", datetime.datetime.now().minute)

now  17 : 29


In [13]:
with open('all_user_event_data.csv', 'w', newline='', encoding='utf8') as f:
    
    writer = csv.writer(f)
    
    writer.writerow(["user_alias", "event_type", "created_at"])
    
    i = 0
    for username in notebook.tqdm(all_username):
        event_url = "https://api.github.com/users/"+username+"/events?per_page=100"
        page1 = []
        page2 = []
        page3 = []
        page1 = CollectUserEventData(event_url+"&page=1")
        if len(page1) == 100:
            page2 = CollectUserEventData(event_url+"&page=2")
            for event in page2:
                page1.append(event)
            if len(page2) == 100:
                page3 = CollectUserEventData(event_url+"&page=3")
                for event in page3:
                    page1.append(event)
        if page1:
            for event in page1:
                writer.writerow([user_alias[i], event[0], event[1]])
        else:
            writer.writerow([user_alias[i], "None", "None"])
        i += 1

        # test
        if i==6:
            break

print("done!")

  0%|          | 0/8583 [00:00<?, ?it/s]

done!


In [14]:
print("now ",datetime.datetime.now().hour,":", datetime.datetime.now().minute)

now  17 : 35
